In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import math
import imageio
import glob2 as glob
import os
import shutil
from skimage.measure import compare_ssim as ssim

In [2]:
pathToVideos = "/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/"

In [3]:
def getAllDirectories (pathToDirectory):
    '''
        Function that gets all the subdirectories of a folder
        
        Args:
            path: Path to root directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    files = glob.glob(pathToDirectory + '*/', recursive=True)
    return files

def getAllFiles (pathToFiles):
    '''
        Function that gets all the filenames from a directory
        
        Args:
            path: Path to directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    print(pathToFiles)
    files = glob.glob(pathToFiles + '*/*/', recursive=True)
    return files

In [25]:
directoriesList = getAllDirectories(pathToVideos)

In [4]:
def extractFramesToFile (pathToFile, dest):

    vidcap = cv2.VideoCapture(pathToFile)    
    success = True
    count = 0
    
    while success:
        success,image = vidcap.read()
        if success:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(dest + "frame" + str(count) + ".jpg", gray)     # save frame as JPEG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
            count += 1
    # When everything done, release the capture
    vidcap.release()
    cv2.destroyAllWindows()

In [27]:
# It was used to get frames from video and save them as images
for path in directoriesList:
    innerFiles = glob.glob(path + '/*')
    for j, innerPath in enumerate(innerFiles):
        folderName = innerPath.rsplit('/', 1)[1][:-4]
        writePath = os.path.join(path, folderName + "/")
        os.mkdir(writePath)
        extractFramesToFile(innerPath, writePath)

In [5]:
filesList = getAllFiles(pathToVideos)

/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/


In [6]:
#Initializing variables

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 10

NUMBER_OF_NP_CANDIDATES = 10

# Scale factor. (prev .2) (pretty good 0.6)
F = 0.9

# Cr probability value. (prev .5) (pretty good 0.9)
Cr = 0.6

In [7]:
# Calculate ASSIM for a chromosome.
def getASSIM(KF, filepath):
    ssim_sum = 0
    for i in range(0, TOTAL_KEY_FRAMES - 2):
        while True:
            try:
                im1 = cv2.imread(filepath + "frame" + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(filepath + "frame" + str(KF[i+1]) + ".jpg",0)
                ssim_sum += ssim(im1, im2)
            except:
#                 print i, KF, KF[i], KF[i+1]
#                 continue
                print (i, KF[i], KF[i+1])
                raise
            break
    return ssim_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and ASSIMs).
def initialize_NP(maxNumberOfFrames, filepath):
    print (maxNumberOfFrames)
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(0, maxNumberOfFrames), TOTAL_KEY_FRAMES)))
        NP[-1].append(getASSIM(NP[-1], filepath))
#         print NP[-1]


# MUTATION
def mutation(parent, maxNumberOfFrames, filepath):
    R = random.sample(NP,2)
    global MV
    MV[:] = []
    MV_value = 0
#     print NP[parent]
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[parent][i] + F*(R[0][i] - R[1][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > maxNumberOfFrames-1):
            MV.append(maxNumberOfFrames-1)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getASSIM(MV, filepath))

    
# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant, filepath):
#     print ("mutant: ", mutant)
#     print ("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getASSIM(TV, filepath))

# SELECTION : Selects offspring / parent based on lower ASSIM value.
def selection(parent, trail_vector):
    if(trail_vector[-1] < parent[-1]):
        parent[:] = trail_vector
#         print ("yes", parent)
#     else:
#         print ("no")

# bestParent returns the parent with then minimum ASSIM value.
def bestParent(population):
    Min_ASSIM_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] < Min_ASSIM_value):
            Min_ASSIM_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index

In [8]:
for i, framesPath in enumerate(filesList):
    # Population matrix.
    NP = []
    # Mutation vector.
    MV = []
    # Trail vector.
    TV = []
    listOfFiles = os.listdir(framesPath) # dir is your directory path
    framesQnt = len(listOfFiles)
    
    print ("Starting routine for file " + framesPath)
    print ("Quantity of frames: " + str(framesQnt))
    
    if (framesQnt < 1):
        print("Corrupted video:" + framesPath)
    else:
        initialize_NP(framesQnt, framesPath)
        for GENERATION in range(STOPPING_ITERATION):
            for i in range(NUMBER_OF_NP_CANDIDATES):
                mutation(i, framesQnt, framesPath)
                crossover(NP[i], MV, framesPath)
                selection(NP[i], TV)
                TV[:] = []
        best_parent = bestParent(NP)
#         print ("best solution is: ", best_parent)

        resultPath = os.path.join(framesPath, "DEResult/")
        os.mkdir(resultPath)
        for frame_number in best_parent[:-1]:
            shutil.copyfile(framesPath + "frame" + str(frame_number) + ".jpg", resultPath + "frame" + str(frame_number) + ".jpg")


Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DESCENDENTE/
Quantity of frames: 67
67
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DISCRIMINACAO/
Quantity of frames: 51
51
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DOZE/
Quantity of frames: 33
33
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DESFIAR/
Quantity of frames: 43
43
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DATA/
Quantity of frames: 34
34
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DISFARCAR/
Quantity of frames: 62
62
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DEPRESSAO/
Quantity of frames: 58
58
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DESFILE/
Quantity of frames: 54
54
Starting routine for file /home/fabiana/Desktop/